## Environment setup & installs

In [3]:
import sys
import subprocess
import importlib
import lpips

def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        print(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir", pkg])

# Core libs
ensure("torch")
ensure("tqdm")
ensure("imageio")
ensure("numpy")
ensure("opencv_python")  # for image resizing if needed
ensure("scikit-image")   # for SSIM
# optional LPIPS (may take time); fallback handled later
try:
    
    _LPIPS_AVAILABLE = True
except Exception:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir", "lpips"])
        _LPIPS_AVAILABLE = True
    except Exception:
        _LPIPS_AVAILABLE = False
        print("lpips not available — LPIPS metric will be skipped.")

# Confirm versions
import torch, numpy as np
print("torch:", torch.__version__, "numpy:", np.__version__, "LPIPS:", _LPIPS_AVAILABLE)

Installing opencv_python ...
Installing scikit-image ...
torch: 2.9.1 numpy: 2.2.6 LPIPS: True


##  Imports, params, positional encoding

In [17]:
import math, os, time
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim
import imageio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


H = 64             
W = 64              
FOV = 40.0          
N_VIEWS = 60        
N_VAL = 10        
NEAR = 0.2
FAR = 2.0
N_SAMPLES = 64       
POS_ENC_L = 10       
DIR_ENC_L = 4        
BATCH_RAYS = 4096   
LR = 5e-4
N_ITERS = 4000


def pos_enc(x, L):
    # x: (..., D)
    enc = [x]
    for i in range(L):
        for fn in [torch.sin, torch.cos]:
            enc.append(fn((2.0**i) * x))
    return torch.cat(enc, dim=-1)

Device: cpu


## Generating synthetic dataset (sphere with Lambert + simple specular)

In [16]:
import cv2

def look_at(cam_pos, target=[0,0,0], up=[0,1,0]):
    cam_pos = np.array(cam_pos)
    target = np.array(target)
    up = np.array(up)
    z = (cam_pos - target)
    z = z / np.linalg.norm(z)
    x = np.cross(up, z)
    x = x / np.linalg.norm(x)
    y = np.cross(z, x)
    R = np.stack([x, y, z], axis=1)  
    T = -R.T @ cam_pos
   
    pose = np.eye(4)
    pose[:3,:3] = R.T
    pose[:3,3] = cam_pos
    return pose 

def render_sphere_image(H, W, cam_pos, f=1.0):
    """Simple analytic renderer of a sphere centered at origin radius=0.5"""
    yy, xx = np.meshgrid(np.linspace(-1,1,H), np.linspace(-1,1,W), indexing='ij')

    dirs = np.stack([xx, -yy, -np.ones_like(xx) * (1.0/f)], axis=-1)
    dirs = dirs / (np.linalg.norm(dirs, axis=-1, keepdims=True) + 1e-9)
    
    cam_to_world = look_at(cam_pos) 
    R = cam_to_world[:3,:3]
    rays_dir_world = dirs.reshape(-1,3) @ R.T
    rays_o = np.tile(np.array(cam_pos).reshape(1,3), (rays_dir_world.shape[0],1))


    C = np.zeros(3)
    r = 0.5
    o = rays_o - C
    d = rays_dir_world
    b = 2.0 * np.sum(o * d, axis=1)
    c = np.sum(o*o, axis=1) - r*r
    disc = b*b - 4*c
    img = np.zeros((H*W,3), dtype=np.float32)
    mask = disc > 0
    t = np.full(H*W, np.inf)
    t[mask] = (-b[mask] - np.sqrt(disc[mask])) / 2.0
    hit = t < np.inf
    pts = rays_o[hit] + d[hit] * t[hit,None]
    normals = (pts - C) / r
   
    light_dir = np.array([1,1,1]); light_dir = light_dir / np.linalg.norm(light_dir)
    lambert = np.clip(np.sum(normals * light_dir, axis=1), 0, 1)
   
    view_dir = -d[hit] / (np.linalg.norm(d[hit],axis=1,keepdims=True)+1e-9)
    halfv = (view_dir + light_dir) / (np.linalg.norm(view_dir + light_dir,axis=1,keepdims=True)+1e-9)
    spec = np.clip(np.sum(normals * halfv, axis=1), 0, 1)**50
    color = (0.7 * lambert + 0.3 * spec)[:,None] * np.array([[0.8,0.3,0.2]])
    img[hit] = color
    img = img.reshape(H,W,3)
    return np.clip(img,0,1)


def gen_dataset(N_views=N_VIEWS, H=H, W=W):
    radius = 1.0
    all_imgs, all_poses = [], []
    angles = np.linspace(0, 2*np.pi, N_views+N_VAL, endpoint=False)
    for i, ang in enumerate(angles):
        cam_pos = [radius * np.cos(ang), 0.0 + 0.1*np.sin(2*ang), radius * np.sin(ang)]
        img = render_sphere_image(H,W, cam_pos, f=1.0)
        pose = look_at(cam_pos)
        all_imgs.append((img*255).astype(np.uint8))
        all_poses.append(pose)
    imgs = np.stack(all_imgs)  
    poses = np.stack(all_poses)  
    return imgs, poses

imgs, poses = gen_dataset()
print("Dataset shapes:", imgs.shape, poses.shape)

os.makedirs("nerf_synth", exist_ok=True)
for i in range(3):
    imageio.imwrite(f"nerf_synth/sample_{i}.png", imgs[i])
print("Saved sample images in ./nerf_synth")

Dataset shapes: (70, 64, 64, 3) (70, 4, 4)
Saved sample images in ./nerf_synth


## Ray Generation and PyTorch Dataset

In [15]:
def get_rays(H, W, focal, c2w):

    """Return rays origins and directions for all pixels in camera c2w (camera-to-world)"""

    i, j = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
    i = i.astype(np.float32); j = j.astype(np.float32)
    
    dirs = np.stack([(i - W*0.5)/focal, -(j - H*0.5)/focal, -np.ones_like(i)], axis=-1)  # (H,W,3)
    
    rays_d = dirs.reshape(-1,3) @ c2w[:3,:3].T
    rays_o = np.broadcast_to(c2w[:3,3], rays_d.shape)
    return rays_o.astype(np.float32), rays_d.astype(np.float32)


class RaysDataset(torch.utils.data.Dataset):

    def __init__(self, imgs, poses, H, W, focal, split="train"):
        self.imgs = imgs
        self.poses = poses
        N = imgs.shape[0]
        split_idx = int(N * 0.8)
        if split == "train":
            self.ids = np.arange(0, split_idx)
        else:
            self.ids = np.arange(split_idx, N)
        self.H, self.W, self.focal = H, W, focal

    def __len__(self):
        return 1000000

    def __getitem__(self, idx):
    
        img_id = np.random.choice(self.ids)
        img = self.imgs[img_id].astype(np.float32)/255.0
        c2w = self.poses[img_id]
        rays_o, rays_d = get_rays(self.H, self.W, self.focal, c2w)
        inds = np.random.choice(self.H*self.W, size=BATCH_RAYS, replace=False)
        rays_o = rays_o[inds]
        rays_d = rays_d[inds]
        pixels = img.reshape(-1,3)[inds]
        return {
            'rays_o': torch.from_numpy(rays_o).float(),
            'rays_d': torch.from_numpy(rays_d).float(),
            'pixels': torch.from_numpy(pixels).float()
        }

focal = 0.5 * W / math.tan(0.5 * math.radians(FOV))
dataset = RaysDataset(imgs, poses, H, W, focal, split="train")
loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=0)
print("Focal:", focal)

Focal: 87.91927742254792


## NeRF MLP

In [13]:
class SimpleNeRF(nn.Module):
    def __init__(self, pos_enc_dim=POS_ENC_L, dir_enc_dim=DIR_ENC_L, W_hidden=256):
        super().__init__()
        in_ch = 3 * (1 + 2*pos_enc_dim)
        dir_ch = 3 * (1 + 2*dir_enc_dim)
        self.fc1 = nn.Sequential(nn.Linear(in_ch, W_hidden), nn.ReLU(),
                                 nn.Linear(W_hidden, W_hidden), nn.ReLU())
        self.fc_sigma = nn.Linear(W_hidden, 1)   
        self.feature_layer = nn.Linear(W_hidden, W_hidden)
        self.color_layer = nn.Sequential(
            nn.Linear(W_hidden + dir_ch, W_hidden//2),
            nn.ReLU(),
            nn.Linear(W_hidden//2, 3),
            nn.Sigmoid()
        )

    def forward(self, x_pts, x_dirs):

        x_pe = pos_enc(x_pts, POS_ENC_L) 
        d_pe = pos_enc(x_dirs / (torch.norm(x_dirs, dim=-1, keepdim=True)+1e-9), DIR_ENC_L)
        h = self.fc1(x_pe)
        sigma = F.relu(self.fc_sigma(h)).squeeze(-1)
        feat = self.feature_layer(h)
        
        color_in = torch.cat([feat, d_pe], dim=-1)
        rgb = self.color_layer(color_in)
        return rgb, sigma

## Sampling and volume rendering

In [12]:
def sample_along_rays(rays_o, rays_d, n_samples=N_SAMPLES, near=NEAR, far=FAR, perturb=True):
    
    R = rays_o.shape[0]
    t_vals = torch.linspace(0.0, 1.0, steps=n_samples, device=rays_o.device)
    z_vals = near * (1 - t_vals) + far * (t_vals)
    z_vals = z_vals.expand([R, n_samples]).clone()
    if perturb:
        mids = 0.5 * (z_vals[:, :-1] + z_vals[:, 1:])
        upper = torch.cat([mids, z_vals[:, -1:]], -1)
        lower = torch.cat([z_vals[:, :1], mids], -1)
        t_rand = torch.rand(z_vals.shape, device=rays_o.device)
        z_vals = lower + (upper - lower) * t_rand
    pts = rays_o.unsqueeze(1) + rays_d.unsqueeze(1) * z_vals.unsqueeze(2)  # (R, n_samples, 3)
    return pts, z_vals

def volume_render_radiance_field(model, rays_o, rays_d, n_samples=N_SAMPLES):
 
    device = rays_o.device
    pts, z_vals = sample_along_rays(rays_o, rays_d, n_samples=n_samples)
    R, S, _ = pts.shape
    pts_flat = pts.reshape(-1,3)
    dirs = rays_d.unsqueeze(1).expand(R, S, 3).reshape(-1,3)

    rgb, sigma = model(pts_flat, dirs)
    rgb = rgb.reshape(R, S, 3)
    sigma = sigma.reshape(R, S)
    
    deltas = z_vals[:,1:] - z_vals[:,:-1]
    delta_last = 1e10 * torch.ones(R,1, device=device)
    deltas = torch.cat([deltas, delta_last], dim=1)  # (R, S)
    alpha = 1.0 - torch.exp(-sigma * deltas)
    trans = torch.cumprod(torch.cat([torch.ones((R,1), device=device), 1.0 - alpha + 1e-10], dim=1), dim=1)[:, :-1]
    weights = alpha * trans  # (R,S)
    rgb_map = torch.sum(weights.unsqueeze(-1) * rgb, dim=1)  # (R,3)
    depth_map = torch.sum(weights * z_vals, dim=1)
    return rgb_map, depth_map, weights

## Training Model

In [ ]:
model = SimpleNeRF().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = None

from itertools import islice
dataloader = iter(loader)

pbar = tqdm(range(N_ITERS), desc="train")
loss_log = []
for it in pbar:
    batch = next(dataloader)
    rays_o = batch['rays_o'].squeeze(0).to(device)  # (B,3)
    rays_d = batch['rays_d'].squeeze(0).to(device)
    pixels = batch['pixels'].squeeze(0).to(device)

    rgb_pred, depth_pred, _ = volume_render_radiance_field(model, rays_o, rays_d, n_samples=N_SAMPLES)
    loss = F.mse_loss(rgb_pred, pixels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_log.append(loss.item())

    if it % 200 == 0 or it==N_ITERS-1:
        
        pbar.set_postfix({'it': it, 'loss': float(loss.item())})

        torch.save(model.state_dict(), "nerf_checkpoint.pth")

## Rendering holdout views & saving as gif

In [ ]:
def render_image(model, c2w, H=H, W=W, focal=focal, nsamples=N_SAMPLES):
    model.eval()
    with torch.no_grad():
        rays_o, rays_d = get_rays(H, W, focal, c2w)
        rays_o = torch.from_numpy(rays_o).float().to(device)
        rays_d = torch.from_numpy(rays_d).float().to(device)
       
        chunk = 8192
        rgb_chunks = []
        for i in range(0, rays_o.shape[0], chunk):
            ro = rays_o[i:i+chunk]
            rd = rays_d[i:i+chunk]
            rgb, depth, _ = volume_render_radiance_field(model, ro, rd, n_samples=nsamples)
            rgb_chunks.append(rgb.cpu().numpy())
        rgb = np.concatenate(rgb_chunks, axis=0)
        img = rgb.reshape(H, W, 3)
        return np.clip(img,0,1)


N = imgs.shape[0]
split_idx = int(N*0.8)
val_ids = np.arange(split_idx, N)
os.makedirs("nerf_renders", exist_ok=True)
frames = []
for vid in val_ids:
    c2w = poses[vid]
    img = render_image(model, c2w)
    gt = imgs[vid].astype(np.float32)/255.0
   
    combined = (np.concatenate([gt, img], axis=1)*255).astype(np.uint8)
    imageio.imwrite(f"nerf_renders/val_{vid}.png", combined)
    frames.append((img*255).astype(np.uint8))

print("Saved renders to nerf_renders/ .")

imageio.mimsave("nerf_renders/novel_views.gif", frames, fps=4)
print("Saved gif: nerf_renders/novel_views.gif")

## Computing metrics

In [ ]:
from skimage import img_as_float32
psnrs, ssims, lpips_vals = [], [], []
lpips_model = None
if _LPIPS_AVAILABLE:
    import lpips
    lpips_model = lpips.LPIPS(net='vgg').to(device)

for vid in val_ids:
    rendered = render_image(model, poses[vid])
    gt = imgs[vid].astype(np.float32)/255.0
    psnrs.append(compute_psnr(gt, rendered, data_range=1.0))

    ssim_val = compute_ssim((gt*255).astype(np.uint8), (rendered*255).astype(np.uint8), multichannel=True, data_range=255)
    ssims.append(ssim_val)
    if lpips_model is not None:

        # LPIPS expects tensors [-1,1]

        with torch.no_grad():
            a = torch.from_numpy((gt*2-1).transpose(2,0,1)).unsqueeze(0).float().to(device)
            b = torch.from_numpy((rendered*2-1).transpose(2,0,1)).unsqueeze(0).float().to(device)
            lp = lpips_model(a, b).item()
        lpips_vals.append(lp)

print("PSNR:", np.mean(psnrs), "SSIM:", np.mean(ssims))
if lpips_model is not None:
    print("LPIPS:", np.mean(lpips_vals))
else:
    print("LPIPS skipped (lpips package unavailable).")

metrics = {"psnr_mean": float(np.mean(psnrs)), "ssim_mean": float(np.mean(ssims)), "lpips_mean": float(np.mean(lpips_vals)) if lpips_vals else None}
import json
with open("nerf_renders/metrics.json","w") as f:
    json.dump(metrics, f, indent=2)
print("Saved metrics to nerf_renders/metrics.json")

## Saving model

In [ ]:
torch.save(model.state_dict(), "nerf_model_final.pth")
print("Saved model: nerf_model_final.pth")

print("GIF saved at nerf_renders/novel_views.gif (use ffmpeg to convert to mp4 if desired)")

## Report

Neural Radiance Fields (NeRF) provide a compact, differentiable representation of 3D appearance by mapping
continuous 3D coordinates and viewing directions to emitted color and volumetric density using a multi-layer
perceptron (MLP). This approach implicitly encodes geometry—rather than storing explicit surfaces—so geometry
emerges from volumetric density predicted across sampled points along camera rays.

This simplified implementation builds an MLP that receives positional encoded 3D coordinates and view
direction encodings. Positional encoding maps low-dimensional inputs into a higher-frequency space using
sine/cosine bases, enabling the network to represent high-frequency variation (sharp edges, fine geometry)
despite using smooth activation functions. The MLP uses a density head (sigma) and a color head; the density
controls opacity along sampled ray segments while the color head conditions on both spatial features and
view direction to reproduce view-dependent effects (specular highlights).

Volume rendering integrates density-weighted colors along a ray: points are sampled between near and far
bounds; network outputs color and density; weights are computed from density and distances between samples;
and the weighted sum yields the pixel color and depth estimate. Training supervises rendered colors against
ground-truth images for many camera poses. The optimization drives the MLP to assign high density where
rays consistently intersect an object's surface thus implicitly reconstructing the geometry, while view-dependent
color allows realistic appearance variations.

Trade-offs: sample count, model width, and training views govern the speed-vs-quality trade-off. More samples
and larger networks improve fidelity but increase compute and memory. Positional encoding frequency boosts
detail but can lead to sneaking high-frequency artifacts or slower convergence. Practical speedups include
coarse-to-fine sampling strategies, hierarchical sampling, smaller inference resolutions, caching learned
features, or using smaller but efficient architectures (e.g., instant-ngp hash encodings or voxel-grid
priors) for real-time rendering. For quality, increasing training views and iterations yields better geometry
and view synthesis.

In this assignment, we used a synthetic sphere dataset to validate the whole pipeline quickly. Despite the
dataset's simplicity compared with complex natural scenes, the model demonstrates how continuous volumetric
representations capture both geometry (density) and appearance (view-conditioned color). The provided code
is modular — replace the synthetic dataset with Blender renders and poses for real scenes. Quantitative metrics
(PSNR/SSIM and LPIPS if available) and saved renders are provided to evaluate and compare different speed/
quality configurations.
